In [1]:
!rm -r book1
!rm *.tar.gz
!wget https://the-eye.eu/public/AI/pile_preliminary_components/books1.tar.gz
!tar -xf books1.tar.gz
path = 'books1/epubtxt/'
selected_book = 'crocodiles-spirit.epub.txt'

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

!pip3 install pytorch-nlp
from torchnlp.word_to_vector import GloVe
glove = GloVe(name='6B', dim='50')

from tqdm.notebook import tqdm
from collections import Counter, defaultdict
import numpy as np
import random
import time
import re

from torch import nn, tensor, optim
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
import torch.nn.functional as F
import torch

import matplotlib.pyplot as plt

rm: cannot remove 'book1': No such file or directory
rm: cannot remove '*.tar.gz': No such file or directory
--2021-05-09 21:31:19--  https://the-eye.eu/public/AI/pile_preliminary_components/books1.tar.gz
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2404269430 (2.2G) [application/octet-stream]
Saving to: ‘books1.tar.gz’

books1.tar.gz       100%[===================>]   2.24G  28.3MB/s    in 2m 22s  

2021-05-09 21:33:42 (16.1 MB/s) - ‘books1.tar.gz’ saved [2404269430/2404269430]

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
     |████████████████████████████████| 90 kB 767 kB/s eta 0:00:01


glove.6B.zip: 862MB [03:34, 4.02MB/s]                               
100%|██████████| 400000/400000 [00:29<00:00, 13620.90it/s]


In [2]:
!ls books1/epubtxt/

1-2-this-is-only-the-beginning.epub.txt
1-god-poems-on-god-creator-volume-1.epub.txt
1-god-poems-on-god-creator-volume-2.epub.txt
1-god-poems-on-god-creator-volume-3.epub.txt
1-god-poems-on-god-creator-volume-4.epub.txt
1-lollapalooza-witness-no-consequence.epub.txt
1-shades-of-gray-noir-city-shrouded-by-darkness-sci-fi-horro.epub.txt
1-shades-of-gray-noir-city-shrouded-by-darkness.epub.txt
10-more-stories.epub.txt
10-of-the-best-stories-from-kenji-miyazawa-and-nankichi-niim.epub.txt
10-reasons-why-the-rapture-must-be-left-behind.epub.txt
10-tales-of-classic-horror.epub.txt
10-ways-to-fk-santa.epub.txt
100-alphanumeric-crosswords.epub.txt
100-incredible-happiness-hacks.epub.txt
100-number-count-crosswords.epub.txt
100-seconds-to-midnight.epub.txt
100-ways-to-kill-your-business.epub.txt
1000-lines-magic-sequence.epub.txt
1000-yards-john-milton-1.epub.txt
1001-changes-to-get-america-back-on-track.epub.txt
1001-sex-positions.epub.txt
101-amazing-things-you-can-do-with-dowsing.epub.txt
101

In [3]:
book1 = 'zombie-school.epub.txt'
book2 = 'a-dream-come-true.epub.txt'
book3 = 'a-lady-out-of-time.epub.txt'

In [4]:
def update_vocab(book, embd, vocab):
    text = re.sub(r"[^a-zA-Z0-9'!?;\.]", r" ", book)
    for sent in sent_tokenize(text):
        words = word_tokenize(sent)
        if len(words) > 100 or len(words) < 3:
            continue
        for word in words:
            word = word.lower()
            if word in embd:
                vocab.add(word)
            else:
                word = word.replace("'","")
                if word in embd:
                    vocab.add(word)

vocab = set(['<s>','</s>','<UKN>'])
with open(path+book1) as file:
    text = file.read()
    update_vocab(text, glove, vocab)

len(vocab)

5462

In [5]:
def sent_preprocess(sent, vocab):
    words = word_tokenize(sent)
    if len(words)>100 or len(words)<3:
        return None
    output = ['<s>']
    for word in words:
        word = word.lower()
        if word not in vocab:
            word = word.replace("'","")
        if word not in vocab:
            word = '<UKN>'
        output.append(word)
    output.append('</s>')
    return output

def preprocess(book, vocab, test_size=0.1):
    text = re.sub(r"[^a-zA-Z0-9'!?;\.]", r" ", book)
    data = list()
    sents = sent_tokenize(text)
    for i,sent in enumerate(tqdm(sents)):
        words = sent_preprocess(sent, vocab)
        if words is None:
            continue
        data.append(words)
    data = np.array(data)
    test_selection = np.zeros(data.shape[0], dtype=bool)
    test_selection[np.random.choice(data.shape[0], data.shape[0]//10, replace=False)] = True
    test = data[test_selection].tolist()
    train = data[~test_selection].tolist()
    return train,test

with open(path+book1) as file:
    text += file.read()+'\n'
train,test = preprocess(text, vocab)
len(train), len(test)

  0%|          | 0/15266 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


(13319, 1479)

In [6]:
def indexing_words(vocabs):
    
    word_to_index = dict()
    index_to_word = dict()
    for i, word in tqdm(enumerate(vocabs)):
        
        word_to_index[word] = i+1
        index_to_word[i+1] = word
        
    word_to_index['<pad>'] = 0
    index_to_word[0] = '<pad>'
    
    return word_to_index, index_to_word

word_to_index, index_to_word = indexing_words(vocab)
vocab.add('<pad>')

0it [00:00, ?it/s]

In [7]:
def replace_words_by_index(data, word_to_index):
    
    new_data = []
    for row in tqdm(data):
        indexed_row = [word_to_index[word] for word in row]
        new_data.append(indexed_row)
        
    return new_data

train = replace_words_by_index(train, word_to_index)
test = replace_words_by_index(test, word_to_index)
len(test), len(train)

  0%|          | 0/13319 [00:00<?, ?it/s]

  0%|          | 0/1479 [00:00<?, ?it/s]

(1479, 13319)

In [8]:
def generate_x_y(data):
    
    x = [tensor(word_list[:-1]) for word_list in data]
    y = [tensor(word_list[1:]) for word_list in data]
    seq_len = [len(word_list)-1 for word_list in data]
            
    return x, y, seq_len

x_train, y_train, seq_len_train = generate_x_y(train)
x_test, y_test, seq_len_test = generate_x_y(test)

In [9]:
test_x_lengths = np.array([len(row) for row in x_test])
train_x_lengths = np.array([len(row) for row in x_train])

test_y_lengths = np.array([len(row) for row in y_test])
train_y_lengths = np.array([len(row) for row in y_train])

x_train = pad_sequence(x_train, batch_first=True, padding_value=word_to_index['<pad>'])
y_train = pad_sequence(y_train, batch_first=True, padding_value=word_to_index['<pad>'])

print(x_train.shape, y_train.shape)

x_test = pad_sequence(x_test, batch_first=True)
y_test = pad_sequence(y_test, batch_first=True)

print(x_test.shape, y_test.shape)

torch.Size([13319, 69]) torch.Size([13319, 69])
torch.Size([1479, 48]) torch.Size([1479, 48])


In [10]:
import numpy as np
def generate_batches(x, y, lengths, batch_size):
    
    indices = [i for i in range(len(x))]
    np.random.shuffle(indices)
    
    try:
        start = 0
        while start < len(x):
            batch_indices = indices[start:start+batch_size]
            yield x[batch_indices], y[batch_indices], lengths[batch_indices]
            start += batch_size
    except:
        batch_indices = indices[start:]
        return x[batch_indices], y[batch_indices], lengths[batch_indices]

In [11]:
class LanguageModel(nn.Module):
    
    def __init__(self, hidden_dim, embedding_dim, n_lsmt_layers, input_dim, output_dim):
        
        super(LanguageModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.n_layers = n_lsmt_layers
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=word_to_index['<pad>'])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_lsmt_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, sequence, hidden, seq_lens=None, eval_=False):
        
        if eval_ == False:
            max_length = len(sequence[0])
            embedded = self.embedding(sequence)
            embedded = pack_padded_sequence(embedded, seq_lens, batch_first=True, enforce_sorted=False)
            lstm_out, hidden = self.lstm(embedded, hidden)
            lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True, total_length=max_length)
            lstm_out = lstm_out.reshape(-1, self.hidden_dim)
            output = self.linear(lstm_out)
        else:
            embedded = self.embedding(sequence)
            lstm_out, hidden = self.lstm(embedded, hidden)
            lstm_out = lstm_out.reshape(-1, self.hidden_dim)
            output = self.linear(lstm_out)
        
        return output, hidden
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        
        return hidden

In [12]:
def train(model, vocab, epochs=100):
        
    losses = []
    batch_loss = None

    loss=nn.CrossEntropyLoss()
    opt=optim.Adam(model.parameters(), lr=0.008)

    model.train()

    for epoch in range(epochs):
        
        print(f'============================== epoch {epoch}, loss {batch_loss}')
        start = time.time()
        
        for x_batch, y_batch, seq_lens in generate_batches(x_train, y_train, train_x_lengths, 256):

            hidden = model.init_hidden(len(x_batch))
            hidden = tuple([h.data for h in hidden])
            
            x_batch = x_batch.cuda()
            y_batch = y_batch.cuda()

            model.zero_grad()

            y_pred, hidden = model(x_batch, hidden, seq_lens=seq_lens)
            batch_loss = loss(y_pred, y_batch.view(-1))
            batch_loss.backward()
            opt.step()

            losses.append(batch_loss)

        print(f'passed time: {time.time() - start}')
        
    return losses

In [13]:
def predict(model, word, hidden, k=3):
    
    x = np.array([[word_to_index[word]]])
    x = torch.from_numpy(x).cuda()
    
    predicted, hidden = model(x, hidden, eval_=True)
    probs = F.softmax(predicted, dim=1).data
    probs = probs.cpu().numpy()
    probs = probs.reshape(probs.shape[1],)
    
    indices = probs.argsort()[-k:][::-1]
    indices = [idx for idx in indices if index_to_word[idx] != '<UKN>' and index_to_word[idx] != '<pad>']
    selected = indices[0]
    
    return index_to_word[selected], hidden


def text_generation(model, init_words, size=50):
    
    model.eval()
    
    hidden = model.init_hidden(1)
    
    word_list = init_words.split()
    for word in word_list:
        predicted, hidden = predict(model, word, hidden)
        
    word_list.append(predicted)
    for i in range(size-1):
        predicted, hidden = predict(model, word_list[-1], hidden)
        word_list.append(predicted)
        if predicted == '</s>':
            break
        
    return word_list

In [14]:
lstm = LanguageModel(200, 100, 3, len(vocab), len(vocab))
lstm.cuda()

losses = train(lstm, vocab, epochs=50)

============================== epoch 0, loss None
passed time: 2.9608004093170166
============================== epoch 1, loss 7.425045490264893
passed time: 2.8435235023498535
============================== epoch 2, loss 6.684749603271484
passed time: 2.7779059410095215
============================== epoch 3, loss 5.906492233276367
passed time: 2.718916177749634
============================== epoch 4, loss 5.290332794189453
passed time: 2.840880870819092
============================== epoch 5, loss 4.749939918518066
passed time: 2.738776922225952
============================== epoch 6, loss 3.856109142303467
passed time: 2.750476121902466
============================== epoch 7, loss 3.200852155685425
passed time: 2.731055498123169
============================== epoch 8, loss 2.7642157077789307
passed time: 2.83703875541687
============================== epoch 9, loss 1.918448567390442
passed time: 2.735499382019043
============================== epoch 10, loss 2.06278657913208
passed 

In [15]:
print(f'from book1\n')

generated = text_generation(lstm, '<s> some times i want to')
print(' '.join(generated))

generated = text_generation(lstm, '<s> here is where')
print(' '.join(generated))

generated = text_generation(lstm, '<s> so')
print(' '.join(generated))

generated = text_generation(lstm, '<s> there are')
print(' '.join(generated))

generated = text_generation(lstm, '<s> i love to')
print(' '.join(generated))

generated = text_generation(lstm, '<s> i like to run far')
print(' '.join(generated))

generated = text_generation(lstm, '<s> in the days of')
print(' '.join(generated))

generated = text_generation(lstm, '<s>')
print(' '.join(generated))

from book1

<s> some times i want to be a zombie i said . </s>
<s> here is where it was a lot and the town 's mentor that it 's a human safe the town 's mentor had been a human . </s>
<s> so you have to be a zombie . </s>
<s> there are n't a zombie i said . </s>
<s> i love to the brain . </s>
<s> i like to run far and do you have to be a zombie i said . </s>
<s> in the days of the stiffs were . </s>
<s> i did n't know that i had n't been a zombie i said . </s>
